In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
import numpy as np
import pickle

def load_data():
    # Specify the file paths for the CIFAR-10 dataset
    data_dir = "cifar-10"
    train_files = ["data_batch_1", "data_batch_2", "data_batch_3", "data_batch_4", "data_batch_5"]
    test_file = "test_batch"
    meta_file = "batches.meta"

    # Function to load a single CIFAR-10 batch file
    def load_cifar_batch(file_path):
        with open(file_path, 'rb') as fo:
            batch_dict = pickle.load(fo, encoding='bytes')
        images = batch_dict[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
        labels = np.array(batch_dict[b'labels'])
        return images, labels

    # Load training data
    train_images = []
    train_labels = []
    for train_file in train_files:
        file_path = f"{data_dir}/{train_file}"
        images, labels = load_cifar_batch(file_path)
        train_images.append(images)
        train_labels.append(labels)

    train_images = np.concatenate(train_images)
    train_labels = np.concatenate(train_labels)

    # Load test data
    test_file_path = f"{data_dir}/{test_file}"
    test_images, test_labels = load_cifar_batch(test_file_path)

    # Load label names
    meta_file_path = f"{data_dir}/{meta_file}"
    with open(meta_file_path, 'rb') as fo:
        label_names = pickle.load(fo, encoding='bytes')[b'label_names']
    
    return train_images, train_labels, test_images, test_labels


In [13]:
# AC-GAN generator
def build_generator(latent_dim, num_classes):
    generator_input = layers.Input(shape=(latent_dim,))
    label_input = layers.Input(shape=(1,))
    label_embedding = layers.Embedding(num_classes, latent_dim)(label_input)
    merged_input = layers.multiply([generator_input, label_embedding])

    x = layers.Dense(256)(merged_input)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(512)(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(1024)(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(3072, activation='tanh')(x)
    x = layers.Reshape((32, 32, 3))(x)

    generator = keras.models.Model([generator_input, label_input], x)
    return generator

# AC-GAN discriminator
def build_discriminator(num_classes):
    discriminator_input = layers.Input(shape=(32, 32, 3))
    label_input = layers.Input(shape=(1,))
    label_embedding = layers.Embedding(num_classes, 32 * 32 * 3)(label_input)
    flat_embedding = layers.Reshape((32, 32, 3))(label_embedding)
    merged_input = layers.concatenate([discriminator_input, flat_embedding])

    x = layers.Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same')(merged_input)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(256)(x)
    x = layers.LeakyReLU()(x)

    discriminator_output = layers.Dense(1, activation='sigmoid')(x)

    discriminator = keras.models.Model([discriminator_input, label_input], discriminator_output)
    discriminator.compile(optimizer=keras.optimizers.Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])
    return discriminator

# AC-GAN combined model
def build_acgan(generator, discriminator):
    generator_input = layers.Input(shape=(latent_dim,))
    label_input = layers.Input(shape=(1,))
    generated_image = generator([generator_input, label_input])
    discriminator_output = discriminator([generated_image, label_input])

    acgan = keras.models.Model([generator_input, label_input], discriminator_output)
    acgan.compile(optimizer=keras.optimizers.Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])
    return acgan

# Training the AC-GAN
latent_dim = 100
num_classes = 10

# Load and preprocess the CIFAR-10 dataset
x_train, y_train, _, _ = load_data()
x_train = x_train.astype('float32') / 255
y_train = keras.utils.to_categorical(y_train, num_classes)

# Build and compile the discriminator
discriminator = build_discriminator(num_classes)
discriminator.trainable = False

# Build the generator
generator = build_generator(latent_dim, num_classes)

# Build and compile the AC-GAN
acgan = build_acgan(generator, discriminator)

# Training loop
epochs = 200
batch_size = 64

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for batch in range(len(x_train) // batch_size):
        # Training the discriminator
        random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
        generated_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
        generated_images = generator.predict([random_latent_vectors, generated_labels])

        real_images = x_train[batch * batch_size: (batch + 1) * batch_size]
        real_labels = y_train[batch * batch_size: (batch + 1) * batch_size]

        discriminator_loss_real = discriminator.train_on_batch([real_images, real_labels], np.ones((batch_size, 1)))
        discriminator_loss_fake = discriminator.train_on_batch([generated_images, generated_labels], np.zeros((batch_size, 1)))
        discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

        # Training the generator
        random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
        generated_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)

        acgan_loss = acgan.train_on_batch([random_latent_vectors, generated_labels], np.ones((batch_size, 1)))

        print(f"Batch {batch+1}/{len(x_train) // batch_size} | D loss: {discriminator_loss[0]:.4f}, G loss: {acgan_loss[0]:.4f}")


Epoch 1/200
2/2 [==============================] - 0s 4ms/step


ValueError: in user code:

    File "d:\Users\Abbas\anaconda3\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "d:\Users\Abbas\anaconda3\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Users\Abbas\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "d:\Users\Abbas\anaconda3\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "d:\Users\Abbas\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer 'reshape_6' (type Reshape).
    
    Cannot reshape a tensor with 1966080 elements to shape [64,32,32,3] (196608 el

In [10]:
y_train.shape

(50000, 10)